In [23]:
import redis
import pickle
import time
import threading
import uuid
import random


class DOSAttackSimulator:
    def __init__(self, node_count, request_count, distance='euclidean'):
        self.redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, password='secret_key')
        self.node_count = node_count
        self.request_count = request_count
        self.query_channel = 'knn_query_channel'
        self.distance = distance
      

    def generate_random_sample(self):
        return [random.uniform(0, 10) for _ in range(64)]

    def send_query(self, distance, voting_method):
        sample = self.generate_random_sample()
        request_id = str(uuid.uuid4())
        query_data = {
            'sample': sample,
            'distance': distance,
            'voting_method': voting_method,
            'request_id': request_id
        }
        self.redis_client.publish(self.query_channel, pickle.dumps(query_data))
        return request_id

    def simulate_attack(self):
        threads = []
        for _ in range(self.request_count):
            t = threading.Thread(target=self.send_query, args=('euclidean','majority_voting'))
            threads.append(t)
            t.start()
            

        for t in threads:
            t.join()
            


node_count = 3  # Number of KNN nodes
request_count = 10000 # Number of requests to send per node

dos_simulator = DOSAttackSimulator(node_count, request_count)


In [22]:
dos_simulator.simulate_attack()

time.sleep(0.01) 